# Data analysis

In [1]:
load("../data/wage2015_subsample_inference.Rdata")
dim(data)

[1] 5150   20

In [2]:
str(data)

'data.frame':	5150 obs. of  20 variables:
 $ wage : num  9.62 48.08 11.06 13.94 28.85 ...
 $ lwage: num  2.26 3.87 2.4 2.63 3.36 ...
 $ sex  : num  1 0 0 1 1 1 1 0 1 1 ...
 $ shs  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ hsg  : num  0 0 1 0 0 0 1 1 1 0 ...
 $ scl  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ clg  : num  1 1 0 0 1 1 0 0 0 1 ...
 $ ad   : num  0 0 0 1 0 0 0 0 0 0 ...
 $ mw   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne   : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1 : num  7 31 18 25 22 1 42 37 31 4 ...
 $ exp2 : num  0.49 9.61 3.24 6.25 4.84 ...
 $ exp3 : num  0.343 29.791 5.832 15.625 10.648 ...
 $ exp4 : num  0.24 92.35 10.5 39.06 23.43 ...
 $ occ  : Factor w/ 369 levels "10","20","40",..: 159 136 269 23 99 86 226 232 184 146 ...
 $ occ2 : Factor w/ 22 levels "1","2","3","4",..: 11 10 19 1 6 5 17 17 13 10 ...
 $ ind  : Factor w/ 236 levels "370","380","390",..: 204 117 12 165 231 176 171 135 210 201 ...
 $ ind2 : Factor w/ 

We are constructing the output variable  𝑌  and the matrix  𝑍  which includes the characteristics of workers that are given in the data.

In [4]:
# selecting a subset of interest (shs and hs)
library(dplyr)
data <-filter(data, shs==1 | hsg==1)
Y <- log(data$wage)
n <- length(Y)
Z <- data[-which(colnames(data) %in% c("wage","lwage"))]
p <- dim(Z)[2]
cat("Number of observation:", n, '\n')
cat( "Number of raw regressors:", p)

Number of observation: 1376 
Number of raw regressors: 18

In [5]:
install.packages("xtable")
library(xtable)
options(xtable.floating = FALSE)
options(xtable.timestamp = "")

package 'xtable' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\gonza\AppData\Local\Temp\Rtmpig0ZWF\downloaded_packages


Warning message:
"package 'xtable' was built under R version 3.6.3"

In [6]:

# keeping variables of interest to do the table
data_z <- data[which(colnames(data) %in% c("lwage","sex","shs","hsg","mw","so","we","ne","exp1"))]




In [7]:
library(xtable)
table <- matrix(0, 9, 1)
table[1:9,1]   <- as.numeric(lapply(data_z,mean))
rownames(table) <- c("Log Wage","Sex","Some High School","High School Graduate","Midwest","South","West","Northeast","Experience" )
colnames(table) <- c("Sample mean")
tab<- xtable(table, digits = 2)
tab

,Sample mean
Log Wage,2.7185624
Sex,0.3219477
Some High School,0.0872093
High School Graduate,0.9127907
Midwest,0.2863372
South,0.2914244
West,0.1984012
Northeast,0.2238372
Experience,17.1900436


E.g., the share of female workers in our sample is ~32% (𝑠𝑒𝑥=1 if female).


# Prediction Question

We construct a prediction rule for hourly wage $Y$ for the people who did not go to college (high school graduates or with some high school education), which depends on job-relevant characteristics $X$:
$$\label{decompose}
Y = \beta'X+ \epsilon.
$$

We aim to predict wages using the exogenous characteristics of workers. For this purpose, we use two different specifications for prediction:

Basic Model: $X$ consists of a set of raw regressors: Sex, Experience, Education, Occupation, Region, etc.

Flexible Model: $X$ consists of all raw regressors from the basic model plus their transformations and two-way interactions. We define these by expressing our set of regressors as:$$\label{regressor}
X=(X_1 + X_2 + X_3+...)^2
$$

In the following sections, we will estimate both models using the OLS method and the Lasso method.

## OLS - BASIC MODEL

In [8]:
basic <- lwage~ sex + exp1 + shs + hsg + scl+clg+ mw + so + we +occ2+ind2
regbasic <- lm(basic, data=data)
regbasic # estimated coefficients
cat( "Number of regressors in the basic model:",length(regbasic$coef), '\n') # number of regressors in the Basic Model


Call:
lm(formula = basic, data = data)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
  2.8330066   -0.0733094    0.0075742   -0.0811342           NA           NA  
        clg           mw           so           we        occ22        occ23  
         NA   -0.0431882   -0.1091620    0.0129620   -0.1961261   -0.0086113  
      occ24        occ25        occ26        occ27        occ28        occ29  
  0.0005078    0.2615289   -0.3510072   -0.1900342   -0.6616521   -0.3013316  
     occ210       occ211       occ212       occ213       occ214       occ215  
 -0.0576220   -0.4176903   -0.4663571   -0.4219896   -0.5527766   -0.4747648  
     occ216       occ217       occ218       occ219       occ220       occ221  
 -0.2381724   -0.3529422   -0.3976108   -0.1181885   -0.1053967   -0.1737437  
     occ222        ind23        ind24        ind25        ind26        ind27  
 -0.3479965    0.1742747    0.0504201    0.0585330    0.0348081    0.2379530

Number of regressors in the basic model: 51 


Note that the flexible model consists of  51 regressors

## OLS - FLEX MODEL

In [9]:
flex <- lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2
regflex <- lm(flex, data=data)
regflex # estimated coefficients
cat( "Number of regressors in the flexible model:",length(regflex$coef)) # number of regressors in the Flexible Model



Call:
lm(formula = flex, data = data)

Coefficients:
  (Intercept)           exp1           exp2           exp3           exp4  
    1.601e+01     -3.411e+00      3.874e+01     -2.133e+01      7.337e+00  
          shs            hsg            scl            clg          occ22  
   -5.723e-01             NA             NA             NA     -2.441e+00  
        occ23          occ24          occ25          occ26          occ27  
   -4.590e+01      7.898e+00     -5.081e+01      1.803e+01     -9.997e-01  
        occ28          occ29         occ210         occ211         occ212  
   -1.024e+01     -2.064e+01     -3.857e+00      2.756e-01      1.765e+00  
       occ213         occ214         occ215         occ216         occ217  
   -2.087e+00     -8.440e-01     -1.059e+01     -1.232e-01     -4.549e+00  
       occ218         occ219         occ220         occ221         occ222  
   -3.445e-01     -6.630e+00     -2.868e+00     -1.239e+00     -1.932e+00  
        ind23          ind24      

Number of regressors in the flexible model: 979

Note that the flexible model consists of  979  regressors

# Try Lasso next 

## Lasso Basic Model

In [10]:
install.packages("hdm")
install.packages("sandwich")
library(hdm)
basic <- lwage~ sex + exp1 + shs + hsg + scl+clg+ mw + so + we +occ2+ind2
lassoregbasic<- rlasso(basic, data=data)

sumlassobasic<- summary(lassoregbasic)

package 'hdm' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\gonza\AppData\Local\Temp\Rtmpig0ZWF\downloaded_packages

  There is a binary version available but the source version is later:
         binary source needs_compilation
sandwich  3.0-0  3.0-1             FALSE



installing the source package 'sandwich'

Warning message:
"package 'hdm' was built under R version 3.6.3"


Call:
rlasso.formula(formula = basic, data = data)

Post-Lasso Estimation:  TRUE 

Total number of variables: 50
Number of selected variables: 6 

Residuals: 
     Min       1Q   Median       3Q      Max 
-1.37681 -0.29491 -0.01412  0.27657  3.47488 

            Estimate
(Intercept)    2.667
sex           -0.098
exp1           0.008
shs            0.000
hsg            0.000
scl            0.000
clg            0.000
mw             0.000
so             0.000
we             0.000
occ22          0.000
occ23          0.000
occ24          0.000
occ25          0.000
occ26          0.000
occ27          0.000
occ28          0.000
occ29          0.000
occ210         0.000
occ211         0.000
occ212         0.000
occ213        -0.240
occ214        -0.312
occ215        -0.302
occ216         0.000
occ217         0.000
occ218         0.000
occ219         0.000
occ220         0.000
occ221         0.000
occ222         0.000
ind23          0.000
ind24          0.000
ind25          0.000
ind26       

## Lasso Flex Model

In [11]:
library(hdm)
flex <- lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2
lassoreg<- rlasso(flex, data=data )

sumlasso<- summary(lassoreg)


Call:
rlasso.formula(formula = flex, data = data)

Post-Lasso Estimation:  TRUE 

Total number of variables: 978
Number of selected variables: 13 

Residuals: 
      Min        1Q    Median        3Q       Max 
-1.584944 -0.306009 -0.009298  0.273618  3.557535 

              Estimate
(Intercept)      2.630
exp1             0.005
exp2             0.000
exp3             0.000
exp4             0.000
shs              0.000
hsg              0.000
scl              0.000
clg              0.000
occ22            0.000
occ23            0.000
occ24            0.000
occ25            0.000
occ26            0.000
occ27            0.000
occ28            0.000
occ29            0.000
occ210           0.000
occ211           0.000
occ212           0.000
occ213          -0.225
occ214          -0.271
occ215           0.000
occ216           0.000
occ217           0.000
occ218           0.000
occ219           0.000
occ220           0.000
occ221           0.000
occ222           0.000
ind23            0.000


Now, we can evaluate the performance of both models based on the (adjusted) $R^2_{sample}$ and the (adjusted) $MSE_{sample}$:

In [12]:
# Assess the predictive performance

sumbasic <- summary(regbasic)
sumflex <- summary(regflex)

#  R-squared 
R2.1 <- sumbasic$r.squared
cat("R-squared for the basic model: ", R2.1, "\n")
R2.adj1 <- sumbasic$adj.r.squared
cat("adjusted R-squared for the basic model: ", R2.adj1, "\n")

R2.2 <- sumflex$r.squared
cat("R-squared for the flexible model: ", R2.2, "\n")
R2.adj2 <- sumflex$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adj2, "\n")

R2.LB <- sumlassobasic$r.squared
cat("R-squared for the lasso with basic model: ", R2.LB, "\n")
R2.adjLB <- sumlassobasic$adj.r.squared
cat("adjusted R-squared for the basic model: ", R2.adjLB, "\n")

R2.L <- sumlasso$r.squared
cat("R-squared for the lasso with flexible model: ", R2.L, "\n")
R2.adjL <- sumlasso$adj.r.squared
cat("adjusted R-squared for the flexible model: ", R2.adjL, "\n")

R-squared for the basic model:  0.1802381 
adjusted R-squared for the basic model:  0.1512255 
R-squared for the flexible model:  0.507044 
adjusted R-squared for the flexible model:  0.2315028 
R-squared for the lasso with basic model:  0.0970881 
adjusted R-squared for the basic model:  0.09313085 
R-squared for the lasso with flexible model:  0.09236528 
adjusted R-squared for the flexible model:  0.0837021 


In [13]:
# calculating the MSE
MSE1 <- mean(sumbasic$res^2)
cat("MSE for the basic model: ", MSE1, "\n")
p1 <- sumbasic$df[1] # number of regressors
MSE.adj1 <- (n/(n-p1))*MSE1
cat("adjusted MSE for the basic model: ", MSE.adj1, "\n")

MSE2 <-mean(sumflex$res^2)
cat("MSE for the flexible model: ", MSE2, "\n")
p2 <- sumflex$df[1]
MSE.adj2 <- (n/(n-p2))*MSE2
cat("adjusted MSE for the flexible model: ", MSE.adj2, "\n")

MSELB <-mean(sumlassobasic$res^2)
cat("MSE for the flexible model: ", MSELB, "\n")
p2 <- sumflex$df[1]
MSE.adjLB <- (n/(n-p2))*MSELB
cat("adjusted MSE for the flexible model: ", MSE.adjLB, "\n")

MSEL <-mean(sumlasso$res^2)
cat("MSE for the lasso flexible model: ", MSEL, "\n")
pL <- length(sumlasso$coef)
MSE.adjL <- (n/(n-pL))*MSEL
cat("adjusted MSE for the lasso flexible model: ", MSE.adjL )

MSE for the basic model:  0.2082191 
adjusted MSE for the basic model:  0.2157451 
MSE for the flexible model:  0.1252106 
adjusted MSE for the flexible model:  0.1953398 
MSE for the flexible model:  0.2293392 
adjusted MSE for the flexible model:  0.3577899 
MSE for the lasso flexible model:  0.2305387 
adjusted MSE for the lasso flexible model:  0.7990461

In [14]:
library(xtable)
table <- matrix(0, 4, 5)
table[1,1:5]   <- c(p1,R2.1,MSE1,R2.adj1,MSE.adj1)
table[2,1:5]   <- c(p2,R2.2,MSE2,R2.adj2,MSE.adj2)
table[3,1:5]   <- c(pL,R2.L,MSEL,R2.adjL,MSE.adjLB)
table[4,1:5]   <- c(pL,R2.L,MSEL,R2.adjL,MSE.adjL)
colnames(table)<- c("p","$R^2_{sample}$","$MSE_{sample}$","$R^2_{adjusted}$", "$MSE_{adjusted}$")
rownames(table)<- c("basic reg","flexible reg", "basic lasso", "flex lasso")
tab<- xtable(table, digits =c(0,0,2,2,2,2))
tab

,p,$R^2_{sample}$,$MSE_{sample}$,$R^2_{adjusted}$,$MSE_{adjusted}$
basic reg,48,0.18023815,0.2082191,0.1512255,0.2157451
flexible reg,494,0.50704400,0.1252106,0.2315028,0.1953398
basic lasso,979,0.09236528,0.2305387,0.0837021,0.3577899
flex lasso,979,0.09236528,0.2305387,0.0837021,0.7990461


# DATA SPLITTING

In [15]:
#splitting the data
set.seed(1) # to make the results replicable (generating random numbers)
random <- sample( 1:n, floor(n*4/5))
# draw (4/5)*n random numbers from 1 to n without replacing them
length(random)

[1] 1100

In [16]:
train <- data[random,] # training sample
test <- data[-random,] # testing sample
dim(train)
dim(test)

[1] 1100   20

[1] 276  20

### Basic Regression

In [17]:

# estimating the parameters in the training sample
regbasic <- lm(basic, data=train)
regbasic


Call:
lm(formula = basic, data = train)

Coefficients:
(Intercept)          sex         exp1          shs          hsg          scl  
   2.836027    -0.113776     0.008171    -0.045630           NA           NA  
        clg           mw           so           we        occ22        occ23  
         NA    -0.007887    -0.092965     0.027930    -0.198916    -0.014664  
      occ24        occ25        occ26        occ27        occ28        occ29  
   0.202306     0.242367     0.006158    -0.180164    -0.796769    -0.241348  
     occ210       occ211       occ212       occ213       occ214       occ215  
  -0.015354    -0.399688    -0.506190    -0.446041    -0.575386    -0.450461  
     occ216       occ217       occ218       occ219       occ220       occ221  
  -0.209308    -0.338949    -0.251586    -0.137541    -0.124913    -0.158960  
     occ222        ind23        ind24        ind25        ind26        ind27  
  -0.350055     0.191507     0.036101     0.034002    -0.004881     0.21484

In [18]:
# calculating the out-of-sample MSE
trainregbasic <- predict(regbasic, newdata=test)
trainregbasic

Warning message in predict.lm(regbasic, newdata = test):
"prediction from a rank-deficient fit may be misleading"

2        4       12       16       18       34       50       52 
2.872798 2.475568 2.909436 2.843801 2.706183 2.816292 3.030303 2.580716 
      53       67       72       76       82       94       98      103 
2.801527 2.787998 3.062171 2.836898 2.326333 2.979703 2.285942 2.276830 
     106      124      137      147      152      154      155      156 
2.412655 2.711248 2.784423 2.636268 2.744333 2.677121 2.840804 3.074304 
     162      163      172      178      184      187      188      189 
2.828920 3.123457 2.388790 2.799951 3.210152 2.792775 2.264362 2.738699 
     195      196      202      203      204      205      212      216 
2.615367 2.817868 2.850550 2.798970 2.905476 2.129962 2.997718 2.490984 
     221      224      227      230      238      239      246      253 
2.297529 2.774117 2.733286 2.812450 2.890822 2.827126 2.760961 2.889827 
     259      274      279      283      297      301      307      319 
2.848974 3.119391 2.354723 2.940427 2.762220 2.724100 2.860109 2.901859 
     333      335      337      350      353      357      366      368 
2.265131 2.460612 2.263418 3.180935 2.501392 2.390470 2.795458 2.496359 
     370      372      373      401      405      409      415      420 
2.809982 2.997905 2.803136 2.738447 2.526222 2.960109 2.769129 2.708267 
     422      424      426      428      429      431      438      440 
2.664675 2.806314 2.703490 2.958472 2.820964 2.734791 2.906452 2.785470 
     445      450      454      468      473      477      479      480 
2.464904 2.993126 2.318446 2.779223 2.921167 2.636552 2.930964 2.704377 
     482      491      496      497      502      506      512      517 
2.421110 2.575738 2.505990 2.752206 2.809982 2.695625 2.825308 2.378906 
     518      527      531      535      540      546      550      551 
2.793640 2.795029 2.962288 2.826964 2.841088 2.933928 2.747153 2.839341 
     561      571      581      585      587      600      602      603 
3.083966 2.808405 2.480362 2.686605 2.844052 2.793640 2.986799 2.733305 
     605      607      612      618      622      625      629      640 
2.569409 2.760837 2.705152 2.948344 2.730090 2.968491 2.502339 2.901246 
     641      647      656      657      662      663      666      667 
2.482359 3.125308 2.160298 2.746005 2.666227 2.802424 2.752024 3.040440 
     673      677      678      680      681      685      687      697 
2.722042 2.407710 2.389074 2.669847 2.595581 2.843988 2.675921 2.824746 
     699      700      703      707      710      711      716      718 
2.876767 2.885334 2.712918 2.734495 2.450420 2.782129 2.572474 2.693666 
     723      726      732      735      736      739      740      757 
2.440899 2.458114 3.381992 2.756010 2.720160 3.187121 2.502988 2.643000 
     759      764      775      784      788      791      796      803 
2.535670 2.380429 2.893551 2.893823 2.723327 2.757586 2.886939 2.268831 
     820      821      826      827      833      834      839      844 
2.326511 2.729321 2.714885 2.658776 2.715157 2.387997 2.578446 2.620074 
     857      859      860      869      876      888      898      901 
2.845393 2.988053 2.934211 2.734891 2.547116 2.409090 2.303548 2.521727 
     904      905      907      911      917      920      926      935 
2.788741 2.619299 2.747839 2.702963 2.651368 2.899300 2.253588 2.704025 
     938      948      951      965      968      978      982     1004 
2.479894 2.623468 2.858322 3.296119 2.892065 2.588540 2.680694 2.521727 
    1006     1013     1016     1029     1030     1035     1050     1055 
2.690374 2.718013 2.293828 2.776616 2.815392 2.704888 2.670543 2.800256 
    1066     1071     1077     1092     1099     1102     1112     1114 
3.132884 2.424292 2.569726 2.293828 1.983952 2.414019 2.866345 2.841260 
    1120     1124     1132     1135     1136     1143     1144     1147 
3.049481 2.589698 2.917757 3.106391 2.950831 2.512371 2.744083 2.750562 
    1149     1157     1164     1166     1168     1172     1176   

In [19]:
y.test <- log(test$wage)
MSE.test1 <- sum((y.test-trainregbasic)^2)/length(y.test)
R2.test1<- 1- MSE.test1/var(y.test)

cat("Test MSE for the basic model: ", MSE.test1, "\n")

cat("Test R2 for the basic model: ", R2.test1)

Test MSE for the basic model:  0.2015247 
Test R2 for the basic model:  0.06784807

In the basic model, the $MSE_{test}$ is quite closed to the $MSE_{sample}$.

### Flexible Regression

In [20]:
# estimating the parameters
#options(warn=-1)
regflex <- lm(flex,data=train)

# calculating the out-of-sample MSE
trainregflex<- predict(regflex, newdata=test )
y.test <- log(test$wage)
MSE.test2 <- sum((y.test-trainregflex)^2)/length(y.test)
R2.test2<- 1- MSE.test2/var(y.test)

cat("Test MSE for the flexible model: ", MSE.test2, "\n")

cat("Test R2 for the flexible model: ", R2.test2)

Warning message in predict.lm(regflex, newdata = test):
"prediction from a rank-deficient fit may be misleading"

Test MSE for the flexible model:  22576.15 
Test R2 for the flexible model:  -104424.9

## Data splitting 


### Lasso - Basic Regression

In [21]:
# Basic model using lasso

# estimating the parameters
library(hdm)
reglassobasic <- rlasso(basic, data=train, post=FALSE)

# calculating the out-of-sample MSE
trainreglassobasic<- predict(reglassobasic, newdata=test)
MSE.lasso1 <- sum((y.test-trainreglassobasic)^2)/length(y.test)
R2.lasso1<- 1- MSE.lasso1/var(y.test)


cat("Test MSE for the lasso on flexible model: ", MSE.lasso1, "\n")

cat("Test R2 for the lasso flexible model: ", R2.lasso1, "\n")

Test MSE for the lasso on flexible model:  0.2051704 
Test R2 for the lasso flexible model:  0.05098462 



### Lasso - Flexible Regression

In [22]:
# flexible model using lasso

# estimating the parameters
library(hdm)
reglassoflex <- rlasso(flex, data=train, post=FALSE)

# calculating the out-of-sample MSE
trainreglassoflex<- predict(reglassoflex, newdata=test)
MSE.lasso2 <- sum((y.test-trainreglassoflex)^2)/length(y.test)
R2.lasso2<- 1- MSE.lasso2/var(y.test)


cat("Test MSE for the lasso on flexible model: ", MSE.lasso2, "\n")

cat("Test R2 for the lasso flexible model: ", R2.lasso2, "\n")

Test MSE for the lasso on flexible model:  0.2023553 
Test R2 for the lasso flexible model:  0.06400605 


In [23]:
table2 <- matrix(0,4,2)
table2[1,1]   <- MSE.test1
table2[2,1]   <- MSE.test2
table2[3,1]   <- MSE.lasso1
table2[4,1]   <- MSE.lasso2
table2[1,2]   <- R2.test1
table2[2,2]   <- R2.test2
table2[3,2]   <- R2.lasso1
table2[4,2]   <- R2.lasso2
rownames(table2)<- c("basic reg","flexible reg"," Basic lasso regression", " Flex lasso regression")
colnames(table2)<- c("$MSE_{test}$", "$R^2_{test}$")
tab2 <- xtable(table2, digits =3)
tab2

,$MSE_{test}$,$R^2_{test}$
basic reg,2.015247e-01,6.784807e-02
flexible reg,2.257615e+04,-1.044249e+05
Basic lasso regression,2.051704e-01,5.098462e-02
Flex lasso regression,2.023553e-01,6.400605e-02


# Partialling-Out using lasso

In [24]:
library(sandwich)

# Model 1 via Lasso
basic.y <- lwage ~  exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2 # model for Y
basic.d <- sex ~ exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2 # model for D

# partialling-out the linear effect of W from Y
t.Y <- rlasso(basic.y, data=data)$res
# partialling-out the linear effect of W from D
t.D <- rlasso(basic.d, data=data)$res

# regression of Y on D after partialling-out the effect of W
partialbasic.lasso.fit <- lm(t.Y~t.D)
partialbasic.lasso.est <- summary(partialbasic.lasso.fit)$coef[2,1]

cat("Coefficient for D via partialling-out using lasso", partialbasic.lasso.est)

# standard error
HCV.coefs <- vcovHC(partialbasic.lasso.fit, type = 'HC')
partialbasic.lasso.se <- sqrt(diag(HCV.coefs))[2]


Coefficient for D via partialling-out using lasso -0.09065628

In [25]:
# Partialling-Out using lasso

#Model 2 via Lasso


# models
extraflex.y <- lwage ~  (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2 # model for Y
extraflex.d <- sex ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)^2 # model for D

# partialling-out the linear effect of W from Y
t.Y <- rlasso(extraflex.y, data=data)$res
# partialling-out the linear effect of W from D
t.D <- rlasso(extraflex.d, data=data)$res

# regression of Y on D after partialling-out the effect of W
partialflex.lasso.fit <- lm(t.Y~t.D)
partialflex.lasso.est <- summary(partialflex.lasso.fit)$coef[2,1]

cat("Coefficient for D via partialling-out using lasso", partialflex.lasso.est)

# standard error
HCV.coefs <- vcovHC(partialflex.lasso.fit, type = 'HC')
partialflex.lasso.se <- sqrt(diag(HCV.coefs))[2]


Coefficient for D via partialling-out using lasso -0.08141371

In [26]:

table<- matrix(0, 2, 2)
table[1,1]<- partialbasic.lasso.est
table[1,2]<- partialbasic.lasso.se    
table[2,1]<-  partialflex.lasso.est
table[2,2]<- partialflex.lasso.se 
colnames(table)<- c("Estimate","Std. Error")
rownames(table)<- c("Basic model via Lasso","Flex model via Lasso")	
tab<- xtable(table, digits=c(3,3,4))
tab

,Estimate,Std. Error
Basic model via Lasso,-0.09065628,0.03223961
Flex model via Lasso,-0.08141371,0.03239641



Both estimators are almost identical and reach the same conclusion: there is a negative wage gap between women and men.